In [1]:
import pandas as pd
import numpy as np
import datetime as dt


from ib_insync import *
import nest_asyncio
nest_asyncio.apply()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=1)

data = {}

In [2]:
util.startLoop()

In [3]:
bank_nifty = Index('BANKNIFTY', 'NSE')
ib.qualifyContracts(bank_nifty)

print(f"bank_nifty's secType : {bank_nifty.secType}")
print(f"bank_nifty's conId   : {bank_nifty.conId}")

bank_nifty's secType : IND
bank_nifty's conId   : 56994300


In [4]:
cds = ib.reqContractDetails(bank_nifty)

len(cds)

1

In [5]:
util.df(cds)

contract marketName  minTick  \
0  Contract(secType='IND', conId=56994300, symbol...                 0.5   

   sizeMinTick                                         orderTypes  \
0          1.0  ACTIVETIM,AD,ADJUST,ALERT,ALLOC,BASKET,BENCHPX...   

  validExchanges  priceMagnifier  underConId    longName contractMonth  ...  \
0            NSE               1           0  Nifty Bank                ...   

  callable putable coupon convertible maturity issueDate nextOptionDate  \
0    False   False      0       False                                     

   nextOptionType  nextOptionPartial  notes  
0                              False         

[1 rows x 42 columns]

In [6]:
ib.reqMarketDataType(4)

In [7]:
[ticker] = ib.reqTickers(bank_nifty)
ticker

Ticker(contract=Index(conId=56994300, symbol='BANKNIFTY', exchange='NSE', currency='INR', localSymbol='BANKNIFTY'), time=datetime.datetime(2022, 9, 29, 8, 41, 39, 228307, tzinfo=datetime.timezone.utc), marketDataType=3, bid=-1.0, bidSize=0.0, ask=-1.0, askSize=0.0, last=37633.5, lastSize=1.0, high=38283.5, low=37546.0, close=37760.0)

In [8]:
chains = ib.reqSecDefOptParams('BANKNIFTY', '', bank_nifty.secType, bank_nifty.conId)
util.df(chains)

exchange underlyingConId tradingClass multiplier  \
0      NSE        56994300    BANKNIFTY          1   

                                         expirations  \
0  [20220929, 20221006, 20221013, 20221020, 20221...   

                                             strikes  
0  [18000.0, 19500.0, 21000.0, 22500.0, 23500.0, ...

In [9]:
chains

[OptionChain(exchange='NSE', underlyingConId='56994300', tradingClass='BANKNIFTY', multiplier='1', expirations=['20220929', '20221006', '20221013', '20221020', '20221027', '20221103', '20221124', '20221229', '20230330', '20230629'], strikes=[18000.0, 19500.0, 21000.0, 22500.0, 23500.0, 24000.0, 24500.0, 25000.0, 25500.0, 26000.0, 26500.0, 27000.0, 27500.0, 28000.0, 28500.0, 29000.0, 29400.0, 29500.0, 29600.0, 29700.0, 29800.0, 29900.0, 30000.0, 30100.0, 30200.0, 30300.0, 30400.0, 30500.0, 30600.0, 30700.0, 30800.0, 30900.0, 31000.0, 31100.0, 31200.0, 31300.0, 31400.0, 31500.0, 31600.0, 31700.0, 31800.0, 31900.0, 32000.0, 32100.0, 32200.0, 32300.0, 32400.0, 32500.0, 32600.0, 32700.0, 32800.0, 32900.0, 33000.0, 33100.0, 33200.0, 33300.0, 33400.0, 33500.0, 33600.0, 33700.0, 33800.0, 33900.0, 34000.0, 34100.0, 34200.0, 34300.0, 34400.0, 34500.0, 34600.0, 34700.0, 34800.0, 34900.0, 35000.0, 35100.0, 35200.0, 35300.0, 35400.0, 35500.0, 35600.0, 35700.0, 35800.0, 35900.0, 36000.0, 36100.0, 36

In [52]:
window = 5

strikes_mid_num = int(len(chains[0].strikes)/2) +1
strike_ATM = chains[0].strikes[strikes_mid_num]

strikes_list = chains[0].strikes[strikes_mid_num - window - 1 : strikes_mid_num + window]

In [101]:
expirations = [chains[0].expirations[0]]
strikes = strikes_list
rights = ['P', 'C']

opt_request_set = [Option('BANKNIFTY', expiration, strike, right, 'NSE', tradingClass='BANKNIFTY')
        for right in rights
        for expiration in expirations
        for strike in strikes]

In [193]:
data['df'] = pd.DataFrame()
data['temp_df'] = pd.DataFrame()


tag_list = ['contract.symbol', 'contract.lastTradeDateOrContractMonth', 'contract.strike', 'contract.right', 'contract.exchange', 'contract.tradingClass', 
            'time', 'marketDataType', 'bid', 'bidSize', 'ask', 'askSize', 'last', 'lastSize', 'prevBid', 'prevBidSize', 'prevAsk', 'prevAskSize', 'prevLast', 
            'volume', 'high', 'low', 'close', 'bidGreeks.tickAttrib', 'bidGreeks.impliedVol', 'bidGreeks.delta', 'bidGreeks.optPrice', 'bidGreeks.pvDividend', 'bidGreeks.gamma', 'bidGreeks.vega', 'bidGreeks.theta','bidGreeks.undPrice', 
'askGreeks.tickAttrib', 'askGreeks.impliedVol', 'askGreeks.delta', 'askGreeks.optPrice', 'askGreeks.pvDividend', 'askGreeks.gamma', 'askGreeks.vega', 'askGreeks.theta','askGreeks.undPrice',
'lastGreeks.tickAttrib', 'lastGreeks.impliedVol', 'lastGreeks.delta', 'lastGreeks.optPrice', 'lastGreeks.pvDividend', 'lastGreeks.gamma', 'lastGreeks.vega', 'lastGreeks.theta', 'lastGreeks.undPrice',
'modelGreeks.tickAttrib', 'modelGreeks.impliedVol', 'modelGreeks.delta', 'modelGreeks.optPrice', 'modelGreeks.pvDividend','modelGreeks.gamma', 'modelGreeks.vega','modelGreeks.theta', 'modelGreeks.undPrice']

tag_list = ['contract.strike', 'contract.right', 'time', 'bid', 'ask']

In [138]:
tickers = ib.reqTickers(*opt_request_set)

In [196]:
for ticker in tickers:
    for tag in tag_list:
        data['temp_df'][tag] = np.nan
        temp_val = eval(f'ticker.{tag}')
        data['temp_df'][tag] = pd.Series(temp_val)
    data['df'] = data['df'].append(data['temp_df'])
    
data['df'] = data['df'].reset_index()    
data['df'] = data['df'].sort_values(by = ['time', 'contract.strike', 'index', 'contract.right'])

cols = ['time', 'contract.strike', 'contract.right',  'bid','ask']
data['df'] = data['df'][cols]

data['df']['spread'] = data['df']['ask'] - data['df']['bid']
data['df']['tick'] = data['df'][['ask', 'bid']].mean(axis = 1)
del data['df']['ask']
del data['df']['bid']

data['df'] = data['df'].drop_duplicates()
data['df']

time  contract.strike contract.right  spread  \
0  2022-09-29 09:57:59.126317+00:00          37000.0              P    0.05   
45 2022-09-29 09:57:59.126317+00:00          37100.0              P    0.05   
46 2022-09-29 09:57:59.126317+00:00          37200.0              P    0.05   
47 2022-09-29 09:57:59.126317+00:00          37300.0              P    0.05   
51 2022-09-29 09:58:00.240079+00:00          37700.0              P    0.30   
55 2022-09-29 09:58:00.241059+00:00          37000.0              C    1.80   
56 2022-09-29 09:58:00.241059+00:00          37100.0              C    1.65   
57 2022-09-29 09:58:00.241059+00:00          37200.0              C    1.00   
58 2022-09-29 09:58:00.241059+00:00          37300.0              C    0.75   
59 2022-09-29 09:58:00.241059+00:00          37400.0              C    0.85   
48 2022-09-29 09:58:00.241059+00:00          37400.0              P    0.05   
60 2022-09-29 09:58:00.241059+00:00          37500.0              C    0.35   
61 2022-09-29 09:58:00.241059+00:00          37600.0              C    0.20   
62 2022-09-29 09:58:00.241059+00:00          37700.0              C    0.05   
63 2022-09-29 09:58:00.241059+00:00          37800.0              C    0.05   
64 2022-09-29 09:58:00.241059+00:00          37900.0              C    0.05   
65 2022-09-29 09:58:00.241059+00:00          38000.0              C    0.05   
49 2022-09-29 09:58:00.242059+00:00          37500.0              P    0.05   
50 2022-09-29 09:58:00.242059+00:00          37600.0              P    0.15   
52 2022-09-29 09:58:00.242059+00:00          37800.0              P    0.25   
53 2022-09-29 09:58:00.242059+00:00          37900.0              P    1.05   
54 2022-09-29 09:58:00.242059+00:00          38000.0              P    0.20   

       tick  
0     0.375  
45    0.325  
46    0.425  
47    0.525  
51  102.200  
55  599.050  
56  498.325  
57  399.100  
58  297.775  
59  198.925  
48    0.625  
60   99.625  
61   18.700  
62    0.925  
63    0.575  
64    0.475  
65    0.475  
49    2.275  
50   20.775  
52  202.225  
53  301.975  
54  401.300

# Notes : 29 Sept 2022
---

1. Draw line charts of different strikes with spread and tick
1. Convert stikes to column names to draw lines